In [16]:
% pip install mesa

In [17]:
# Se importan las clases de la librería MESA, Agent (para los agentes) y Model (entorno)
from mesa import Agent, Model 
from mesa.space import MultiGrid # Como se requieren varios agentes por celda importamos «MultiGrid»
from mesa.time import SimultaneousActivation # SimultanousActivation permite que los agentes se activen al mismo tiempo
from mesa.datacollection import DataCollector #DataCollector nos arroja información de los pasos de la simulación

# Para crear una animación de cada paso del modelo 
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Librerías para el manejo de números
import numpy as np
import pandas as pd
import random

# Librerías para medición del tiempo
import time
import datetime

In [18]:
""" Construimos la habitación, se define estado de los diferentes agentes
    Se pide el modelo de donde se obtendrá la información
    Devuelve la matriz con la información de los estados de los agentes"""
def get_habitacion(model):
    habitacion = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for data_content in content:
            if isinstance(data_content, Robot):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = data_content.state
    return habitacion

""" Agente de cada celda o parte del piso"""
""" Se define el estado que puede ser CLEAN o DIRTY"""
class Cell(Agent):
    
    CLEAN = 0
    DIRTY = 1
    
    def __init__(self, pos, model, state = CLEAN):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state

""" Agente «Robot», establece su actual y próximo movimiento
    Si la celda está sucia la va a limpiar, de lo contrario
    se moverá a una de sus 8 celdas vecinas de forma aleatoria"""
class Robot(Agent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.newPosition = None
        self.moves = 0
    
    def step(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = True)
        
        for neighbour in neighbors:
            if neighbour.pos == self.pos:
                if neighbour.state == neighbour.DIRTY:
                    neighbour.state = neighbour.CLEAN
                    self.newPosition = self.pos
                else:
                    neighborhood = self.model.grid.get_neighborhood(
                        self.pos,
                        moore = True,
                        include_center = False)
                    newPosition = self.random.choice(neighborhood)
                    self.newPosition = newPosition
                break
        
        """Se cuentan los movimientos"""
        if self.pos != self.newPosition:
            self.moves += 1
        
        self.model.grid.move_agent(self, self.newPosition)

""" Modelo de la habitación, establecemos parámetros de sus agentes y entorno"""
class Habitacion(Model):
    
    def __init__(self, m, n, agentNumber, dirtyCellPercentage):
        self.agentNumber = agentNumber
        self.dirtyCellPercentage = dirtyCellPercentage
        self.cleanPercentage = 1 - dirtyCellPercentage
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
        
        """Lllenamos de forma aleatoria las celdas sucias necesarias en la habitación"""
        dirty_cells = int((m * n) * dirtyCellPercentage)
        emptyCellsList = list(self.grid.empties)
        
        for cells in range(dirty_cells):
            empty_cell = random.choice(emptyCellsList)
            cell = Cell(empty_cell, self)
            cell.state = cell.DIRTY
            self.grid.place_agent(cell, empty_cell)
            self.schedule.add(cell)
            emptyCellsList.remove(empty_cell)
        
        """Consideramos limpias al resto de las celdas"""
        emptyCellsList = list(self.grid.empties)
        for cells in emptyCellsList:
            cell = Cell(cells, self)
            self.grid.place_agent(cell, cells)
            self.schedule.add(cell)
        
        """Los agentes robots se posicionan en la celda [1,1]"""
        for i in range(agentNumber):
            robot = Robot(i, self)
            self.grid.place_agent(robot, (1,1))
            self.schedule.add(robot)
        
        """Se guardan los datos"""
        self.data_collector = DataCollector(
            model_reporters={'Habitación': get_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)})
        
    def step(self):
        self.data_collector.collect(self)
        self.schedule.step()    
    
    """Cálculo de porcentaje de limpieza"""
    def cleanAllCells(self):
        cleanCells = 0
        for cell in self.grid.coord_iter():
            cellContent, x, y = cell
            for content in cellContent:
                if isinstance(content, Cell) and content.state == content.CLEAN:
                    cleanCells += 1
        
        self.cleanPercentage = cleanCells / (self.grid.width * self.grid.height)
        if self.cleanPercentage == 1:
            return True
        else:
            return False

In [19]:
# Dimensiones de la habitación
M = 10
N = 70

# Número de agentes
Agent_Number = 50

# Porcentaje de celdas sucias en decimal
Dirty_Cells_Percentage = 0.4

# Tiempo máximo de ejecución (segundos)
Max_Execution_Time = 3

# Inicialización del tiempo
start_timer = time.time()

# Construcción del modelo
model = Habitacion(M, N, Agent_Number, Dirty_Cells_Percentage)

# Ejecución de la simulación
while((time.time() - start_timer) < Max_Execution_Time and not model.cleanAllCells()):
    model.step()

# TIempo de ejecución de la simulación
Execution_Time = str(datetime.timedelta(seconds=(time.time() - start_timer)))

In [20]:
# Obtenemos la información de la simulación para visualizarla
habitaciones = model.data_collector.get_model_vars_dataframe()

In [21]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(habitaciones.iloc[0][0], cmap='Pastel1')

def animate(i):
    patch.set_data(habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(habitaciones))

In [23]:
print('Dimensión de la habitación: ', M, ' x ', N)
print('Número de robots: ', Agent_Number)
print('Porcentaje de celdas sucias: ', Dirty_Cells_Percentage*100, '%')
print('Tiempo máximo de ejecución: ', Max_Execution_Time, 's')
print('Tiempo necesario de limpieza:', Execution_Time)
print('Porcentaje de celdas limpias al concluir:', model.cleanPercentage*100, '%')

moves = model.data_collector.get_agent_vars_dataframe()
print('Número de movimientos relizados por los agentes:', moves.tail()['Movimientos'].sum())
anim

Dimensión de la habitación:  10  x  70
Número de robots:  50
Porcentaje de celdas sucias:  40.0 %
Tiempo máximo de ejecución:  3 s
Tiempo necesario de limpieza: 0:00:01.956311
Porcentaje de celdas limpias al concluir: 100.0 %
Número de movimientos relizados por los agentes: 0
